In [1]:
!pip install -q lxml

In [2]:
import bs4
import lxml
import pandas as pd
import urllib
import numpy

In [3]:
from urllib import request

In [10]:
def firefoxPage(link):
    req = request.Request(
        url=link, 
        headers={'User-Agent': 'Mozilla/5.0'} # IMPORTANT (sinon connection refusee)
    )
    html_page = request.urlopen(req)
    return html_page

### Obtenir les liens de tous les joueurs en LCK summer 2023

In [46]:
from bs4 import BeautifulSoup

# https://gol.gg/players/list/season-ALL/split-ALL/tournament-LCK%20Summer%202022/

LCK_split_link_list = [("https://gol.gg/players/list/season-ALL/split-ALL/tournament-LCK%20Summer%202023/", 2023, "Summer"),
                       ("https://gol.gg/players/list/season-ALL/split-ALL/tournament-LCK%20Spring%202023/", 2023, "Spring"),
                       ("https://gol.gg/players/list/season-ALL/split-ALL/tournament-LCK%20Summer%202022/", 2022, "Summer"),
                       ("https://gol.gg/players/list/season-ALL/split-ALL/tournament-LCK%20Spring%202022/", 2022, "Spring"),
                       ("https://gol.gg/players/list/season-ALL/split-ALL/tournament-LCK%20Summer%202021/", 2021, "Summer"),
                       ("https://gol.gg/players/list/season-ALL/split-ALL/tournament-LCK%20Spring%202021/", 2021, "Spring"),
                       ("https://gol.gg/players/list/season-ALL/split-ALL/tournament-LCK%20Summer%202020/", 2020, "Summer"),
                       ("https://gol.gg/players/list/season-ALL/split-ALL/tournament-LCK%20Spring%202020/", 2020, "Spring")]

player_link_list = []
for (link, year, split) in LCK_split_link_list:
    html_page = firefoxPage(link)
    soup = BeautifulSoup(html_page, features="lxml")
    for link in soup.find_all('a'):
        if str(link.get('href')).startswith("./player-stats/"):
            player_link_list.append((str(link.get('href')).strip('.'), year, split))

### Obtenir les stats des joueurs

In [57]:
def checkCols(cols):
    if cols == None:
        return False
    for k in cols:
        if k != '':
            return True
    return False

data = []
columnNames = ['Player', 'Year', 'Split']
generateCol = True
for link, year, split in player_link_list:
    link = link.strip()
    html = firefoxPage("https://gol.gg/players" + link)
    soup = BeautifulSoup(html, features="lxml")

    table = soup.find("table", attrs={"class": "table_list"})
    name = soup.find("h1").text.strip()
    playerStat = [name, year, split]

    rows = table.find_all('tr')
    if columnNames != ['Player', 'Year', 'Split']:
        generateCol = False
    for row in rows:

        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]

        if checkCols(cols):
            if generateCol:
                columnNames.append(cols[0])
            playerStat = playerStat + [elem for elem in cols[1:] if elem != '']
    data.append(playerStat)

df = pd.DataFrame(data, columns=columnNames)
df.to_csv("LCK_player_stats.csv", index=False)



In [64]:
df = pd.read_csv("LCK_player_stats.csv")

### Obtenir les liens de tous les equipes

In [50]:
from bs4 import BeautifulSoup

def LCK_link_suffix(split, year):
    
    return ('LCK ' + split + ' ' + str(year) + '/','LCK%20' + split + '%20' + str(year) + '/')

LCK_split_link_list = [("https://gol.gg/teams/list/season-S13/split-Summer/tournament-LCK%20Summer%202023/", 2023, "Summer"),
                       ("https://gol.gg/teams/list/season-S13/split-Spring/tournament-LCK%20Spring%202023/", 2023, "Spring"),
                       ("https://gol.gg/teams/list/season-S12/split-ALL/tournament-LCK%20Summer%202022/", 2022, "Summer"),
                       ("https://gol.gg/teams/list/season-S12/split-Spring/tournament-LCK%20Spring%202022/", 2022, "Spring"),
                       ("https://gol.gg/teams/list/season-S11/split-ALL/tournament-LCK%20Summer%202021/", 2021, "Summer"),
                       ("https://gol.gg/teams/list/season-S11/split-Spring/tournament-LCK%20Spring%202021/", 2021, "Spring"),
                       ("https://gol.gg/teams/list/season-S10/split-ALL/tournament-LCK%20Summer%202020/", 2020, "Summer"),
                       ("https://gol.gg/teams/list/season-S10/split-Spring/tournament-LCK%20Spring%202020/", 2020, "Spring")]

team_link_list = []

for (link, year, split) in LCK_split_link_list:
    html_page = firefoxPage(link)
    soup = BeautifulSoup(html_page, features="lxml")
    for link2 in soup.find_all('a'):
        if str(link2.get('href')).startswith("./team-stats/"):
            old, new = LCK_link_suffix(split, year)
            team_link_list.append((str(link2.get('href')).strip('.').replace(old, new), year, split))

### Obtenir les roles des joueurs

In [51]:
def get_player_table(soup, team_name):
    tables = soup.find_all('table')
    for table in tables:
        caption = table.find('caption')
        if caption and caption.text.strip() == team_name + ' player\'s stats':
            return table
        else:
            continue
    return None

labels = ['Team', 'Year', 'Split','Role','Player']
data = []

for link, year, split in team_link_list:
    link = link.strip()
    html = firefoxPage("https://gol.gg/teams" + link)
    soup = BeautifulSoup(html, features="lxml")

    table = soup.find("table", attrs={"class": "table_list"})
    name = soup.find("h1").text.strip()
    rows = get_player_table(soup, name).find_all('tr')
    
    for row in rows:
        tds = row.find_all('td')
        if len(tds) >= 2:
            stat = [name] # Team
            stat.append(year) # Year
            stat.append(split) # Split
            stat.append(tds[0].text.strip())  # Role
            stat.append(tds[1].text.strip()) # Player
            data.append(stat)
dfrole = pd.DataFrame(data, columns=labels)

In [52]:
print(dfrole)

      Team  Year   Split     Role  Player
0    BRION  2023  Summer      TOP  Morgan
1    BRION  2023  Summer   JUNGLE    UmTi
2    BRION  2023  Summer      MID   Karis
3    BRION  2023  Summer      BOT    Hena
4    BRION  2023  Summer  SUPPORT  Effort
..     ...   ...     ...      ...     ...
634     T1  2020  Spring  SUPPORT  Effort
635     T1  2020  Spring      TOP   Roach
636     T1  2020  Spring   JUNGLE   Ellim
637     T1  2020  Spring      BOT  Effort
638     T1  2020  Spring  SUPPORT   Teddy

[639 rows x 5 columns]


In [53]:
dfrole.to_csv("LCK_player_role.csv", index=False)

### Stats finaux joueurs

In [66]:
players = pd.merge(dfrole, df, on=['Player', 'Year', 'Split'])

In [65]:
print(df)

     Player  Year   Split    Record: Win Rate: KDA:  CS per Minute:  \
0    Aiming  2023  Summer   34W - 6L       85%  9.1            10.5   
1     Andil  2023  Summer   2W - 18L       10%  1.3             1.1   
2       Bdd  2023  Summer   34W - 6L       85%  7.2             8.8   
3     BeryL  2023  Summer  16W - 27L     37.2%  2.1             1.3   
4     Bible  2023  Summer    8W - 5L     61.5%  2.8             1.1   
..      ...   ...     ...        ...       ...  ...             ...   
543   Viper  2020  Spring  16W - 29L     35.6%  3.9             9.9   
544    Vsta  2020  Spring  13W - 22L     37.1%  3.4             8.3   
545   Zenit  2020  Spring    1W - 4L       20%  1.6             8.0   
546    bono  2020  Spring  24W - 17L     58.5%  4.1             5.2   
547    kurO  2020  Spring  24W - 22L     52.2%  4.7             8.3   

     Gold Per Minute: Gold%: Kill Participation:  
0                 508    26%               75.2%  
1                 215  12.7%               64

In [67]:
print(players)

      Team  Year   Split     Role  Player    Record: Win Rate: KDA:  \
0    BRION  2023  Summer      TOP  Morgan  15W - 30L     33.3%  2.1   
1    BRION  2023  Summer   JUNGLE    UmTi  15W - 30L     33.3%  2.9   
2    BRION  2023  Summer      MID   Karis  12W - 23L     34.3%  3.5   
3    BRION  2023  Summer      BOT    Hena  15W - 30L     33.3%  3.3   
4    BRION  2023  Summer  SUPPORT  Effort  15W - 30L     33.3%  2.6   
..     ...   ...     ...      ...     ...        ...       ...  ...   
634     T1  2020  Spring  SUPPORT   Teddy  30W - 14L     68.2%  7.1   
635     T1  2020  Spring  SUPPORT  Effort  30W - 14L     68.2%  3.6   
636     T1  2020  Spring      BOT  Effort  30W - 14L     68.2%  3.6   
637     T1  2020  Spring      TOP   Roach    2W - 2L       50%  2.1   
638     T1  2020  Spring   JUNGLE   Ellim    6W - 3L     66.7%  6.3   

     CS per Minute:  Gold Per Minute: Gold%: Kill Participation:  
0               8.9               390  22.3%                 52%  
1            

### Obtenir stats equipes

In [68]:
from bs4 import BeautifulSoup

data = []
labels = None
for (link, year, split) in LCK_split_link_list:
    html_page = firefoxPage(link)
    soup = BeautifulSoup(html_page, features="lxml")
    table = soup.find_all('table')[1]
    rows = table.find_all('tr')
    if labels == None:
        labels = ['Year', 'Split', 'Team'] + [elem.text.strip() for elem in table.find('thead').find_all('th')]
    for row in rows:
        stat = [year, split]
        tds = row.find_all('td')
        for td in tds:
            stat.append(td.text.strip())
        if (stat != []):
            data.append(stat)

dfteam  = pd.DataFrame(data, columns=labels)

ValueError: 33 columns passed, passed data had 30 columns

In [204]:
print(dfteam)

                  Name Season Region Games Win rate   K:D   GPM   GDM  \
0                BRION    S13     KR    45    33.3%  0.79  1749  -112   
1            Dplus KIA    S13     KR    40    60.0%  1.23  1859    78   
2                  DRX    S13     KR    43    37.2%  0.72  1733  -159   
3        Gen.G eSports    S13     KR    39    82.1%  1.77  1969   275   
4  Hanwha Life eSports    S13     KR    41    65.9%  1.27  1887    98   
5           KT Rolster    S13     KR    41    85.4%  1.94  1957   248   
6     Kwangdong Freecs    S13     KR    38    26.3%  0.61  1746  -164   
7         Liiv SANDBOX    S13     KR    40    32.5%  0.82  1784  -106   
8    Nongshim RedForce    S13     KR    43    30.2%  0.70  1739  -190   
9                   T1    S13     KR    42    50.0%  0.97  1860    60   

  Game duration Kills / game  ... TD@15  GD@15   PPG NASHPG NASH%   CSM   DPM  \
0         32:17          8.4  ...   0.0   -277  3.47   0.60  43.8  34.5  1848   
1         31:38         13.1  ... 